In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.patches import Polygon
from os import listdir
%matplotlib inline
from itertools import combinations

Sto plottando semplicemente il numero di hit vs i relativi conteggi

In [ ]:
for i in listdir("/Users/gianmarco/OneDrive/UniPd/Magistrale/I Anno/Laboratory of Computational Physics/data/calibration/file uniti"):
    file="/Users/gianmarco/OneDrive/UniPd/Magistrale/I Anno/Laboratory of Computational Physics/data/calibration/file uniti"+"/"+i
    with open(file) as f:
        hit_num=[]
        for line in f:
            event = line.split()
            event = [float(i) for i in event]
            hit_num.append(event[1])
        eventi_tot=len(hit_num)+1
        eventi_buoni=0
        for j in hit_num: 
            if j==0: #qua voglio togliere da eventi_tot tutti gli eventi che hanno 0 hit
                eventi_tot-=1
            elif (j==6 or j==7 or j==8 or j==9 or j==10): #qua conto solo quelli che hanno 6/7/8/9/10 hit
                eventi_buoni+=1
    print("eventi totali:",eventi_tot)
    print("eventi buoni:",eventi_buoni)
    print("% dati buoni",i, ":", eventi_buoni/eventi_tot*100, "%")
    print(len(hit_num))
    bins=int(max(hit_num))+1
    #bins=[i for i in range(0,15)] #per plottare solo i dati più interessanti: avere ≈80 bin è inutile
    print("numero bins",bins)
    plt.figure()
    plt.hist(hit_num,bins)
    plt.xlabel("# of hit")
    plt.ylabel("counts")
    plt.title(i) 
    plt.grid(True)
    titolo=i+".png"
    plt.savefig(titolo)

In [2]:
# Cell dimensions
XCELL = 42.
ZCELL = 13.

# X coordinates translation
global_x_shifts = [994.2, 947.4,-267.4,-261.5,]

# Z coordinates translations
local_z_shifts = [z*ZCELL for z in range(0,4)]
global_z_shifts = [823.5, 0, 823.5, 0]

In [ ]:
for i in listdir("/Users/gianmarco/OneDrive/UniPd/Magistrale/I Anno/Laboratory of Computational Physics/data/calibration/file uniti"):
    file="/Users/gianmarco/OneDrive/UniPd/Magistrale/I Anno/Laboratory of Computational Physics/data/calibration/file uniti"+"/"+i
    with open(file) as f:
        for line in f:
            event = line.split()
            event = [float(i) for i in event]
            event_number = int(event[0])
            hits_number  = int(event[1])
            hit       = np.arange(hits_number)
            chamber   = np.fromiter((event[2+5*i] for i in range(hits_number)), int)
            layer     = np.fromiter((event[3+5*i] for i in range(hits_number)), int)
            xl_local  = np.fromiter((event[4+5*i] for i in range(hits_number)), float)
            xr_local  = np.fromiter((event[5+5*i] for i in range(hits_number)), float)
            z_local   = np.fromiter((local_z_shifts[i-1]+ZCELL/2 for i in layer), float)
            time      = np.fromiter((event[6+5*i] for i in range(hits_number)), float)
            xl_global = np.fromiter((global_x_shifts[i] for i in chamber), float) - xl_local
            xr_global = np.fromiter((global_x_shifts[i] for i in chamber), float) - xr_local
            z_global  = np.fromiter((global_z_shifts[i] for i in chamber), float) + z_local
            dataframe = pd.DataFrame(
                { 'EvNumber' : event_number,
                  'Hit'      : hit,
                  'Chamber'  : chamber,
                  'Layer'    : layer,
                  'XL_local' : xl_local,
                  'XR_local' : xr_local,
                  'Z_local'  : z_local,
                  'Time'     : time,
                  'XL_global': xl_global,
                  'XR_global': xr_global,
                  'Z_global' : z_global,
                })
            #dataframe.set_index('Hit', inplace=True) # set as index the number of the hit
            #print(dataframe)
            for j in range(0,4):
                dataframe1=dataframe.loc[dataframe['Chamber'] == j] #in questo modo ottengo un dataframe per ogni evento e chamber
                #print(dataframe1)
                unici=dataframe1['Layer'].nunique() 
                if unici>=3: #mi assicuro che ci siano almeno 3 hit in 3 layer diversi
                    xl=np.array(dataframe1['XL_local'])
                    #print(xl)
                    xr=np.array(dataframe1['XR_local'])
                    #print(xr)
                    z=np.array(dataframe1['Z_local'])
                    #print(z)
                    
                    puntir=[]
                    for j in range(0,np.shape(z)[0]):
                        puntir.append((z[j],xr[j]))
                    #print(puntir)
                    puntil=[]
                    for j in range(0,np.shape(z)[0]):
                        puntil.append((z[j],xl[j]))
                    #print(puntil)
                    layer1=[]
                    layer2=[]
                    layer3=[]
                    layer4=[]
                    for j in puntil:
                        if j[0]==6.5:
                            layer1.append(j)
                        elif j[0]==19.5:
                            layer2.append(j)
                        elif j[0]==32.5:
                            layer3.append(j)
                        elif j[0]==45.5:
                            layer4.append(j)
                    for j in puntir:
                        if j[0]==6.5:
                            layer1.append(j)
                        elif j[0]==19.5:
                            layer2.append(j)
                        elif j[0]==32.5:
                            layer3.append(j)
                        elif j[0]==45.5:
                            layer4.append(j)
                    #print(layer1)
                    #print(layer2)
                    #print(layer3)
                    #print(layer4)
                    if unici==3: #PER CREARE LE COMBINAZIONI CON 3 ELEMENTI
                        if len(layer1)==0:
                            combinazioni=np.array([(x,y,z) for x in layer2 for y in layer3 for z in layer4])
                        elif len(layer2)==0:
                            combinazioni=np.array([(x,y,z) for x in layer1 for y in layer3 for z in layer4])
                        elif len(layer3)==0:
                            combinazioni=np.array([(x,y,z) for x in layer1 for y in layer2 for z in layer4])
                        elif len(layer4)==0:
                            combinazioni=np.array([(x,y,z) for x in layer1 for y in layer2 for z in layer3])
                        #print(combinazioni)
                        chimin=1000
                        punti=0
                        coeff=np.zeros((2, 3))
                        for i in combinazioni: #PER OGNI COMBINAZIONE FACCIO IL FIT, CALCOLO IL CHI^2 E PLOTTO SOLO LA COMBINAZIONE CON IL CHI MINORE
                            p=np.polyfit(i[:,0], i[:,1], 1) #calcolo coeff ZANETTI AVEVA DETTO DI FARE x(z)
                            #print("coefficienti fit:",p)
                            chi_squared=np.sum(((np.polyval(p, i[:,0]) - i[:,1]) ** 2)) #calcolo chi2
                            #print("chiquadro:",chi_squared)
                            if chi_squared<chimin: #trovo il chi minore e salvo la combianzione e i parametri ad esso associato
                                chimin=chi_squared
                                punti=i
                                coeff=p
                                print(coeff)
                        print("coefficiente migliore:", coeff)
                        #print(type(coeff), type(coeff.item(0)), type(coeff.item(0)))
                        print("Chi^2 migliore:",chimin)
                        x=np.linspace(0,50,50)
                        y=coeff.item(0)*x + coeff.item(1)
                        #print("retta interpolante:",y)
                        plt.figure()
                        plt.plot(punti[:,1], punti[:,0],"o",label='original data')
                        plt.plot(y,x,"-",label="fitted data")
                        plt.xlabel("x [mm]")
                        plt.ylabel("z [mm]")
                        plt.title(punti) #DA SISTEMARE IL TITOLO    
                        plt.grid(True)
                        plt.show()
                    elif unici==4: #PER CREARE LE COMBINAZIONI CON 4 ELEMENTI
                        combinazioni=np.array([(x,y,z,w) for x in layer1 for y in layer2 for z in layer3 for w in layer4])
                        #print(combinazioni)
                        chimin=1000
                        punti=0
                        coeff=coeff=np.zeros((2, 4))
                        for i in combinazioni: #PER OGNI COMBINAZIONE FACCIO IL FIT, CALCOLO IL CHI^2 E PLOTTO SOLO LA COMBINAZIONE CON IL CHI MINORE
                            p=np.polyfit(i[:,0], i[:,1], 1) #calcolo coeff
                            #print("coefficienti fit:",p)
                            chi_squared=np.sum(((np.polyval(p, i[:,0]) - i[:,1]) ** 2)) #calcolo chi2
                            #print("chiquadro:",chi_squared)
                            if chi_squared<chimin: #trovo il chi minore e salvo la combianzione e i parametri ad esso associato
                                chimin=chi_squared
                                punti=i
                                coeff=p
                        #print("coefficiente migliore:", coeff)
                        #print(type(coeff), type(coeff[0]), type(coeff[1]))
                        print("Chi^2 migliore:",chimin)
                        x=np.linspace(0,50,50)
                        y=coeff[0]*x + coeff[1]
                        #print("retta interpolante:",y)
                        plt.figure()
                        plt.plot(punti[:,1], punti[:,0],"o",label='original data')
                        plt.plot(y,x,"-",label="fitted data")
                        plt.xlabel("x [mm]")
                        plt.ylabel("z [mm]")
                        plt.title(punti) #DA SISTEMARE IL TITOLO    
                        plt.grid(True)
                        plt.show()
                        

# Funzioni

In [3]:
#Read events
def read_data(event):    
    event_number = int(event[0])
    hits_number  = int(event[1])
    hit       = np.arange(hits_number)
    chamber   = np.fromiter((event[2+5*i] for i in range(hits_number)), int)
    layer     = np.fromiter((event[3+5*i] for i in range(hits_number)), int)
    xl_local  = np.fromiter((event[4+5*i] for i in range(hits_number)), float)
    xr_local  = np.fromiter((event[5+5*i] for i in range(hits_number)), float)
    z_local   = np.fromiter((local_z_shifts[i-1]+ZCELL/2 for i in layer), float)
    time      = np.fromiter((event[6+5*i] for i in range(hits_number)), float)
    xl_global = np.fromiter((global_x_shifts[i] for i in chamber), float) - xl_local
    xr_global = np.fromiter((global_x_shifts[i] for i in chamber), float) - xr_local
    z_global  = np.fromiter((global_z_shifts[i] for i in chamber), float) + z_local
    dataframe = pd.DataFrame(
        { 'EvNumber' : event_number,
          'Hit'      : hit,
          'Chamber'  : chamber,
          'Layer'    : layer,
          'XL_local' : xl_local,
          'XR_local' : xr_local,
          'Z_local'  : z_local,
          'Time'     : time,
          'XL_global': xl_global,
          'XR_global': xr_global,
          'Z_global' : z_global,
        })
    #dataframe.set_index('Hit', inplace=True) # set as index the number of the hit 
    return dataframe, event_number, hits_number




#Select events
def select_events(dataframe):
        
        #hits only in the right side
        if((dataframe['Chamber']<=1).all()):
            chambers=[0,1]
            #compute number of different layers in each chamber
            n_layer_ch0 = dataframe[dataframe['Chamber']==0]['Layer'].nunique()
            n_layer_ch1 = dataframe[dataframe['Chamber']==1]['Layer'].nunique()
            
            n_layer=[n_layer_ch0, n_layer_ch1]
            
            #require at least 3 different layers for each chamber
            if(n_layer_ch0>=3 and n_layer_ch1>=3):
                select=True
                return select, chambers, n_layer
            else:
                select=False
                return select, chambers, n_layer
            
        #hits only in the left side
        elif((dataframe['Chamber']>=2).all()):
            chambers=[2,3]
            #compute number of different layers in each chamber
            n_layer_ch2 = dataframe[dataframe['Chamber']==2]['Layer'].nunique()
            n_layer_ch3 = dataframe[dataframe['Chamber']==3]['Layer'].nunique() 
            
            n_layer=[n_layer_ch2, n_layer_ch3]
            
            #require at least 3 different layers for each chamber
            if(n_layer_ch2>=3 and n_layer_ch3>=3):
                select=True
                return select, chambers, n_layer
            else:
                select=False
                return select, chambers, n_layer
        
        #hits in both left and right side
        else:
            select=False
            chambers=[]
            n_layer=[]
            return select, chambers, n_layer 
        
        
path= "../data/calibration/Run000260/"      
data_file = path+"data_000000.txt"
with open(data_file) as f:
    line = f.readline()
    line = f.readline()
    line = f.readline()
    line = f.readline()
    line = f.readline()
    event = line.split()
    #event = [float(i) for i in event]
    print(event)
    
    
ev, evNumber, hits = read_data(event)    

['5', '11', '2', '2', '7.695e+01', '9.105e+01', '130.8', '2', '3', '4.967e+01', '7.633e+01', '247.5', '2', '4', '7.494e+01', '9.306e+01', '168.3', '2', '1', '4.833e+01', '7.767e+01', '272.5', '3', '4', '4.160e+01', '4.240e+01', '7.5', '3', '3', '1.869e+02', '1.911e+02', '39.2', '3', '2', '4.065e+01', '4.335e+01', '25.0', '3', '1', '1.421e+02', '1.519e+02', '91.7', '3', '4', '1.532e+02', '1.828e+02', '275.0', '3', '1', '4.362e+01', '8.238e+01', '360.0', '3', '3', '4.312e+01', '8.288e+01', '369.2']


# Local Linear Fit

In [29]:
import random

def local_fit(dataframe, list_chambers, list_layers):
    #list to store results for each chamber
    results=[]
    #loop over the two chambers
    for i in range(0,len(list_chambers)):
       #if we have 4 different layers we randomly select a layer to be excluded
       #we will use the point from the excluded layer to check the goodness of the global fit
        if(list_layers[i]==4):
            rand_layer=random.randint(1,4)
        else:
            rand_layer=0 #layers are 1,2,3,4: excluding layer 0 is equivalent to keeping them all
        
        #create dataframe_cl filtered by chamber and excluded layer
        dataframe_c = dataframe[dataframe['Chamber']==list_chambers[i]] #dataframe filtered by chamber
        dataframe_cl = dataframe_c[dataframe_c['Layer']!=rand_layer]    #filtered by chamber and excluded layer
        
        # Z local coordinates corresponding to the 4 different layers
        Z=[6.5,19.5, 32.5, 45.5]
        
        #create a list l containing 3 lists of points (z,x), one for each selected layer
        l=[]
        
        #loop over selected layers and fill l
        for layer_index in dataframe_cl['Layer'].unique():
            XR=np.array(dataframe_cl[dataframe_cl['Layer']==layer_index]['XR_local'])
            XL=np.array(dataframe_cl[dataframe_cl['Layer']==layer_index]['XL_local'])
            
            z=Z[(layer_index-1)] #layer_index is in range [1,4], list index must be in range [0,3]
            l_temp=[]
            
            for x in XR:
                l_temp.append((z,x))
            for x in XL:
                l_temp.append((z,x)) 
            l.append(l_temp) 
            
        #create numpy array with all possible combinations of 3 points p1,p2,p3
        combinations=np.array([(p1,p2,p3) for p1 in l[0] for p2 in l[1] for p3 in l[2]])
        
        #interpolate each combination and select the combination with least chi squared
        min_chisq=100000 #to store minimum chisq
        optimal_comb=np.zeros((3,2)) #to store best combination of points
        for points in combinations:
            #linear regression
            slope, intercept, r_value, p_value, std_err=stats.linregress(points[:,0],points[:,1])
            #compute expected x using the interpolating function
            expect_x=intercept+slope*(points[:,0])
            #compute chi squared
            chisq, p_value=stats.chisquare(points[:,1],expect_x)
            #eventually update min_chisq and optimal_comb
            if(chisq<min_chisq):
                min_chisq=chisq
                optimal_comb=points
            else:
                continue
        #add to results: results is a list of 2 dictionaries, one for each chamber       
        results.append({"slope":slope, 
                   "intercept":intercept, 
                   "optimal_comb": optimal_comb, 
                   "excl_layer": rand_layer})
                   
    return results
        
        

    
    
print(ev)    
condition, chambers, n_layers=select_events(ev)    
if(condition==True):
    lf_results=local_fit(ev, chambers, n_layers)
    print("Prima camera\n",lf_results[0]["optimal_comb"])
    print("Seconda camera\n", lf_results[1])
    

    EvNumber  Hit  Chamber  Layer  XL_local  XR_local  Z_local   Time  \
0          5    0        2      2     76.95     91.05     19.5  130.8   
1          5    1        2      3     49.67     76.33     32.5  247.5   
2          5    2        2      4     74.94     93.06     45.5  168.3   
3          5    3        2      1     48.33     77.67      6.5  272.5   
4          5    4        3      4     41.60     42.40     45.5    7.5   
5          5    5        3      3    186.90    191.10     32.5   39.2   
6          5    6        3      2     40.65     43.35     19.5   25.0   
7          5    7        3      1    142.10    151.90      6.5   91.7   
8          5    8        3      4    153.20    182.80     45.5  275.0   
9          5    9        3      1     43.62     82.38      6.5  360.0   
10         5   10        3      3     43.12     82.88     32.5  369.2   

    XL_global  XR_global  Z_global  
0     -344.35    -358.45     843.0  
1     -317.07    -343.73     856.0  
2     -342.3